In [1]:
import pandas as pd

## Total nodes from lcc files

In [ ]:
nodes = pd.read_csv("../../data/processed/finals/lcc_nodes.tsv", sep="\t", low_memory=False)

category = pd.value_counts(nodes["category"])

prefixes= pd.value_counts(nodes["id"].str.split(":").str[0])

print("Nodes by category: \n")
display(category)
print("Nodes by prefix: \n")
display(prefixes)

## Total edges from lcc files

In [ ]:
edges = pd.read_csv("../../data/processed/finals/lcc_edges.tsv", sep="\t", low_memory=False)

predicate = pd.value_counts(edges["predicate"])

print("Edges by predicate: \n")
display(predicate)

## Create pivot table for Sankey Graph from lcc file

In [ ]:
smalledges = edges[["subject", "object"]]
smalledges["subject"] = smalledges["subject"].str.split(":").str[0]
smalledges["object"] = smalledges["object"].str.split(":").str[0]
pivot = smalledges.pivot_table(columns=['subject','object'], aggfunc='size')

pivot

## Analyse node categories and prefixes from lcc merge file by source

In [ ]:
nodes["source"] = nodes["source"].str.split("|")

nodes = nodes.explode(["source"])
nodes["prefixes"] = nodes.id.str.split(":").str[0]

categories = nodes[["source", "category"]].groupby(["category","source"]).size().unstack(fill_value=0)

prefixes = nodes[["source", "prefixes"]].groupby(["prefixes","source"]).size().unstack(fill_value=0)

display(categories)
display(prefixes)

## Analyse edges predicates from lcc merge file by source

In [ ]:
edges["source"] = edges["source"].str.split("|")

edges = edges.explode(["source"])

predicates = edges[["source", "predicate"]].groupby(["predicate","source"]).size().unstack(fill_value=0)

predicates.to_csv("lcc_predicates.tsv", sep="\t")
display(predicates)

## Create table with category-predicate-category by source

In [ ]:
filterededges = edges[
        ["subject", "object", "predicate", "source"]]
filterednodes = nodes[
        ["id", "category"]].drop_duplicates().set_index("id")

filterednodes = filterednodes[~filterednodes.index.duplicated(keep="first")].iloc[:, 0]

filterededges["subject category"] = filterededges["subject"].map(filterednodes)
filterededges["object category"] = filterededges["object"].map(filterednodes)
filterededges["edge"] = filterededges["subject category"] + "-" + filterededges["predicate"] + "-" + filterededges["object category"]

filterededges = filterededges[["source", "edge"]]
filterededges["source"] = filterededges["source"].str.split("|")
filterededges = filterededges.explode(["source"])

completetable = filterededges.groupby(["edge", "source"]).size().unstack(fill_value=0)
completetable